<a href="https://colab.research.google.com/github/roccaab/WaveletGAN/blob/main/Tesi_Presentazione.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pywavelets

# Generazione di Accelerogrammi Sintetici con CGAN e Decomposizione Wavelet

# 1. Installazione delle dipendenze
!pip install numpy pandas matplotlib torch pywavelets requests

# 2. Importazioni
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pywt
import requests
import zipfile
import io
import json
import time
import random
from google.colab import files

# 3. Configurazione
# @title Configurazione del progetto
DATA_DIR = "./data"  # @param {type:"string"}
OUTPUT_DIR = "./output"  # @param {type:"string"}
USE_GPU = True  # @param {type:"boolean"}

# Crea le directory di lavoro
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(os.path.join(DATA_DIR, "raw"), exist_ok=True)
os.makedirs(os.path.join(DATA_DIR, "normalized"), exist_ok=True)
os.makedirs(os.path.join(DATA_DIR, "json"), exist_ok=True)  # Corretto questa riga
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "models"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "images"), exist_ok=True)

# Imposta device (CPU/GPU)
device = torch.device("cuda" if USE_GPU and torch.cuda.is_available() else "cpu")
print(f"Dispositivo utilizzato: {device}")

# Imposta device (CPU/GPU)
device = torch.device("cuda" if USE_GPU and torch.cuda.is_available() else "cpu")
print(f"Dispositivo utilizzato: {device}")

# 4. Funzioni di utilità per la gestione dei dati
# @title Funzioni per l'acquisizione e preprocessing dei dati sismici

def normalize_data(data):
    """
    Normalizza i dati dell'accelerogramma

    Args:
        data: Array numpy con i dati dell'accelerogramma

    Returns:
        Array numpy con i dati normalizzati
    """
    # Sottrazione della media
    data_mean = np.mean(data)
    data_centered = data - data_mean

    # Divisione per deviazione standard
    data_std = np.std(data_centered)
    if data_std > 0:
        data_normalized = data_centered / data_std
    else:
        data_normalized = data_centered  # Evita divisione per zero

    return data_normalized

def process_accelerogram(zip_ref, filename, aid, mw, epic_dist, component, output_dir):
    """
    Processa un singolo file di accelerogramma

    Args:
        zip_ref: Riferimento all'archivio ZIP
        filename: Nome del file nell'archivio
        aid: ID dell'accelerogramma
        mw: Magnitudo
        epic_dist: Distanza epicentrale
        component: Componente (E o N)
        output_dir: Directory di output
    """
    print(f"Elaborazione file {filename}, AID={aid}")

    # Legge il contenuto del file
    with zip_ref.open(filename) as f:
        content = f.read().decode('utf-8', errors='replace').splitlines()

    # Trova la fine dell'intestazione (dove iniziano i dati numerici)
    data_start = 0
    pga_value = None

    # Cerca il PGA nei metadati
    for i, line in enumerate(content):
        if "PGA_CM/S^2:" in line:
            parts = line.split(":")
            if len(parts) > 1:
                try:
                    pga_value = abs(float(parts[1].strip()))
                    print(f"Trovato PGA nei metadati: {pga_value} cm/s²")
                except ValueError:
                    pass

        # Trova l'inizio dei dati numerici
        try:
            float(line.strip())
            data_start = i
            break
        except ValueError:
            continue

    # Estrae solo i dati numerici (questo rimuove i metadati iniziali)
    data_lines = content[data_start:]

    # Stampa informazioni sulla rimozione dei metadati
    print(f"Rimossi {data_start} righe di metadati dal file {filename}")

    # Converte in array di float
    try:
        acceleration_data = np.array([float(line.strip()) for line in data_lines])
    except ValueError as e:
        print(f"Errore nella conversione dei dati: {str(e)}")
        return

    # Se il PGA non è stato trovato nei metadati, calcolalo dai dati
    if pga_value is None:
        pga_value = np.max(np.abs(acceleration_data))
        print(f"PGA calcolato dai dati: {pga_value} cm/s²")

    # Estrai 10 secondi attorno al picco PGA (5 secondi prima e 5 secondi dopo)
    fs = 200  # Frequenza di campionamento (Hz)
    peak_idx = np.argmax(np.abs(acceleration_data))

    # Calcola gli indici per la finestra di 10 secondi
    start_idx = max(0, peak_idx - 5 * fs)
    end_idx = min(len(acceleration_data), peak_idx + 5 * fs)

    # Se la finestra è più corta di 10 secondi, adatta gli indici
    if (end_idx - start_idx) < 10 * fs:
        if start_idx == 0:  # Vicino all'inizio
            end_idx = min(len(acceleration_data), start_idx + 10 * fs)
        else:  # Vicino alla fine
            start_idx = max(0, end_idx - 10 * fs)

    # Estrai la finestra di 10 secondi
    window_data = acceleration_data[start_idx:end_idx]

    # Se la finestra non è esattamente 2000 campioni, ricampiona o taglia
    if len(window_data) != 2000:
        if len(window_data) > 2000:
            # Taglia
            excess = len(window_data) - 2000
            window_data = window_data[excess//2:excess//2+2000]
        else:
            # Estendi con zeri
            pad_width = 2000 - len(window_data)
            window_data = np.pad(window_data, (pad_width//2, pad_width - pad_width//2), 'constant')

    # Normalizza i dati: sottrazione della media e divisione per deviazione standard
    normalized_data = normalize_data(window_data)

    # Salva il file normalizzato
    normalized_file = os.path.join(output_dir, "normalized", f"{aid}.nrm")
    np.savetxt(normalized_file, normalized_data, fmt='%.8f')

    # Crea e salva il file JSON con i metadati
    metadata = {
        "AID": aid,
        "MW": float(mw),
        "epic_dist": float(epic_dist),
        "PGA": float(pga_value),
        "component": component,
        "original_file": filename
    }

    json_file = os.path.join(output_dir, "json", f"{aid}.json")
    with open(json_file, 'w') as f:
        json.dump(metadata, f, indent=4)

    print(f"Salvato accelerogramma AID={aid}, componente {component}, PGA={pga_value}")

def download_and_process_data(csv_path, output_dir="./data"):
    """
    Scarica e processa i dati sismici dal portale INGV (progetto ITACA)

    Args:
        csv_path: Percorso del file CSV con i dati degli eventi
        output_dir: Directory di output per i file elaborati
    """
    # Crea directory di output se non esistono
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, "raw"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "normalized"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "json"), exist_ok=True)

    # Carica il CSV con i dati degli eventi usando il separatore corretto (punto e virgola)
    df = pd.read_csv(csv_path, sep=';')

    # Stampa le colonne del CSV per il debug
    print("Colonne disponibili nel CSV:", df.columns.tolist())

    print(f"Trovati {len(df)} eventi nel file CSV")

    aid_counter = 1  # Contatore per l'ID degli accelerogrammi

    # Itera sulle righe del CSV
    for idx, row in df.iterrows():
        try:
            # Estrae i valori usando i nomi corretti delle colonne
            event_id = row['esm_event_id']  # o 'ingv_event_id' se disponibile
            station = row['station_code']
            mw = row['mw']
            epic_dist = row['epi_dist']  # o 'proximity' se disponibile

            print(f"Elaborazione evento {event_id}, stazione {station} ({idx+1}/{len(df)})")

            # Costruisce l'URL dell'API INGV
            api_url = f"https://itaca.mi.ingv.it/itaca40ws/eventdata/1/query?eventid={event_id}&station={station}&format=ascii"

            try:
                # Scarica il file ZIP
                response = requests.get(api_url)
                response.raise_for_status()  # Solleva eccezione per errori HTTP

                # Salva il file ZIP temporaneamente
                temp_zip_path = os.path.join(output_dir, "raw", f"{event_id}_{station}.zip")
                with open(temp_zip_path, 'wb') as f:
                    f.write(response.content)

                print(f"File ZIP scaricato: {temp_zip_path}")

                # Estrai e processa i file dal ZIP
                with zipfile.ZipFile(temp_zip_path, 'r') as zip_ref:
                    # Lista tutti i file nell'archivio
                    file_list = zip_ref.namelist()

                    # Filtra i file per le componenti E e N
                    e_files = [f for f in file_list if 'HGE' in f or 'HNE' in f]
                    n_files = [f for f in file_list if 'HGN' in f or 'HNN' in f]

                    # Processa i file componente E
                    for e_file in e_files:
                        process_accelerogram(zip_ref, e_file, aid_counter, mw, epic_dist, 'E', output_dir)
                        aid_counter += 1

                    # Processa i file componente N
                    for n_file in n_files:
                        process_accelerogram(zip_ref, n_file, aid_counter, mw, epic_dist, 'N', output_dir)
                        aid_counter += 1

            except Exception as e:
                print(f"Errore durante l'elaborazione dell'evento {event_id}: {str(e)}")
                continue
        except KeyError as e:
            print(f"Errore nell'accesso alla colonna: {str(e)} per la riga {idx+1}")
            continue

    print(f"Elaborazione completata. Totale accelerogrammi elaborati: {aid_counter-1}")

# 5. Funzioni per la decomposizione wavelet
# @title Funzioni per la decomposizione wavelet

def wavelet_decomposition(signal, wavelet='db4', level=6):
    """
    Scompone un segnale in componenti wavelet usando la trasformata wavelet discreta (DWT)

    Args:
        signal: Segnale da decomporre (array 1D)
        wavelet: Famiglia wavelet da utilizzare (default: 'db4')
        level: Livello di decomposizione (default: 6)

    Returns:
        Lista di componenti wavelet [A6, D1, D2, D3, D4, D5, D6]
        dove A6 è l'approssimazione di livello 6 e Di sono i dettagli
    """
    # Verifica che il segnale sia un array 1D
    if isinstance(signal, list):
        signal = np.array(signal)

    if len(signal.shape) > 1:
        signal = signal.flatten()

    # Applica la decomposizione wavelet
    coeffs = pywt.wavedec(signal, wavelet, level=level)

    # Estrai approssimazione (A6) e dettagli (D1-D6)
    components = []

    # Costruisci una lista di coefficienti per ogni componente
    for i in range(len(coeffs)):
        # Crea una copia dei coefficienti con tutti zero tranne l'i-esimo
        coeff_i = []
        for j in range(len(coeffs)):
            if j == i:
                coeff_i.append(coeffs[j])
            else:
                coeff_i.append(np.zeros_like(coeffs[j]))

        # Ricostruisci il segnale con solo l'i-esimo coefficiente
        rec = pywt.waverec(coeff_i, wavelet)

        # Taglia alla lunghezza originale
        if len(rec) > len(signal):
            rec = rec[:len(signal)]
        elif len(rec) < len(signal):
            # Questo non dovrebbe accadere, ma gestiamo il caso per sicurezza
            rec = np.pad(rec, (0, len(signal) - len(rec)), 'constant')

        components.append(rec)

    # Verifica che tutte le componenti abbiano la stessa lunghezza
    target_length = 2000
    for i in range(len(components)):
        if len(components[i]) < target_length:
            # Padding
            components[i] = np.pad(components[i], (0, target_length - len(components[i])), 'constant')
        elif len(components[i]) > target_length:
            # Troncamento
            components[i] = components[i][:target_length]

    return components

def get_component_energy(components):
    """
    Calcola l'energia di ciascuna componente wavelet

    Args:
        components: Lista di componenti wavelet [A6, D1, D2, D3, D4, D5, D6]

    Returns:
        Array di energie normalizzate (somma a 1)
    """
    # Calcola l'energia di ciascuna componente
    energies = np.array([np.sum(comp**2) for comp in components])

    # Normalizza le energie
    total_energy = np.sum(energies)
    if total_energy > 0:
        normalized_energies = energies / total_energy
    else:
        normalized_energies = np.ones_like(energies) / len(energies)

    return normalized_energies

def plot_wavelet_decomposition(signal, components, save_path=None):
    """
    Visualizza la decomposizione wavelet di un segnale

    Args:
        signal: Segnale originale
        components: Lista di componenti wavelet [A6, D1, D2, D3, D4, D5, D6]
        save_path: Percorso dove salvare l'immagine (opzionale)
    """
    labels = ['A6', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6']
    n_components = len(components)

    # Crea la figura
    fig, axs = plt.subplots(n_components + 1, 1, figsize=(12, 2.5 * (n_components + 1)))

    # Plot del segnale originale
    t = np.arange(len(signal)) / 200  # Assume 200 Hz sampling rate
    axs[0].plot(t, signal)
    axs[0].set_title('Segnale originale')
    axs[0].set_ylabel('Ampiezza')

    # Plot delle componenti
    for i in range(n_components):
        if i < len(labels):
            label = labels[i]
        else:
            label = f'Componente {i}'

        t_comp = np.arange(len(components[i])) / 200
        axs[i+1].plot(t_comp, components[i])
        axs[i+1].set_title(f'Componente {label}')
        axs[i+1].set_ylabel('Ampiezza')

    axs[-1].set_xlabel('Tempo (s)')

    plt.tight_layout()

    # Salva o mostra
    if save_path:
        plt.savefig(save_path)
        plt.close()
    else:
        plt.show()

def process_all_signals(data_dir, output_dir=None, wavelet='db4', level=6, plot=False):
    """
    Processa tutti i segnali normalizzati e genera la decomposizione wavelet

    Args:
        data_dir: Directory contenente i file .nrm
        output_dir: Directory di output per le componenti wavelet
        wavelet: Famiglia wavelet da utilizzare
        level: Livello di decomposizione
        plot: Se True, genera e salva i plot
    """
    # Se non specificato, usa sottodirectory della directory dati
    if output_dir is None:
        output_dir = os.path.join(data_dir, "wavelet")

    # Crea directory di output se non esiste
    os.makedirs(output_dir, exist_ok=True)

    # Directory per i plot
    if plot:
        plot_dir = os.path.join(output_dir, "plots")
        os.makedirs(plot_dir, exist_ok=True)

    # Trova tutti i file .nrm
    normalized_dir = os.path.join(data_dir, "normalized")
    nrm_files = [f for f in os.listdir(normalized_dir) if f.endswith('.nrm')]

    print(f"Trovati {len(nrm_files)} file normalizzati da processare")

    component_energies = {}

    # Processa ogni file
    for nrm_file in nrm_files:
        aid = os.path.splitext(nrm_file)[0]
        print(f"Elaborazione decomposizione wavelet per AID={aid}")

        try:
            # Carica il segnale normalizzato
            signal_path = os.path.join(normalized_dir, nrm_file)
            signal = np.loadtxt(signal_path)

            # Applica la decomposizione wavelet
            components = wavelet_decomposition(signal, wavelet, level)

            # Calcola l'energia delle componenti
            energies = get_component_energy(components)
            component_energies[aid] = energies.tolist()  # Converti in lista per JSON

            # Salva ogni componente in un file separato
            comp_names = ['A6'] + [f'D{i}' for i in range(1, level + 1)]
            for i, comp in enumerate(components):
                if i < len(comp_names):
                    comp_name = comp_names[i]
                else:
                    comp_name = f"Comp{i}"

                comp_path = os.path.join(output_dir, f"{aid}_{comp_name}.npy")
                np.save(comp_path, comp)

            # Genera e salva il plot
            if plot:
                plot_path = os.path.join(plot_dir, f"{aid}_wavelet.png")
                plot_wavelet_decomposition(signal, components, plot_path)

        except Exception as e:
            print(f"Errore durante l'elaborazione di AID={aid}: {str(e)}")
            import traceback
            traceback.print_exc()
            continue

    # Salva le energie delle componenti
    energies_path = os.path.join(output_dir, "component_energies.json")
    with open(energies_path, 'w') as f:
        json.dump(component_energies, f, indent=4)

    print("Elaborazione completata")

# 6. Definizione dei modelli CGAN
# @title Modelli CGAN (Generator e Discriminator)

def weights_init(m):
    """
    Inizializzazione dei pesi per migliorare la convergenza delle GAN

    Args:
        m: Modulo PyTorch
    """
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
    elif classname.find('Linear') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

def textToVect(metadata):
    """
    Converte i metadati in un vettore di condizionamento

    Args:
        metadata: Dizionario con MW (magnitudo) e PGA

    Returns:
        Vettore NumPy di dimensione 2 con [MW normalizzata, PGA normalizzata]
    """
    # Estrazione dei valori
    mw = float(metadata['MW'])
    pga = float(metadata['PGA'])

    # Normalizzazione empirica basata sui dati tipici
    # Magnitudo: 4.0-7.0 -> [-1, 1]
    norm_mw = 2 * (mw - 4.0) / 3.0 - 1

    # PGA: fino a 1000 cm/s^2 -> [-1, 1]
    norm_pga = 2 * (pga / 1000.0) - 1

    # Clamp nei range validi
    norm_mw = max(-1, min(1, norm_mw))
    norm_pga = max(-1, min(1, norm_pga))

    return np.array([norm_mw, norm_pga], dtype=np.float32)

class Generator(nn.Module):
    """
    Generatore per la CGAN

    Input:
    - Vettore latente z (dimensione: latent_dim)
    - Informazioni condizionali (dimensione: cond_dim)

    Output:
    - Accelerogramma sintetico (dimensione: 2000 campioni)
    """
    def __init__(self, latent_dim=100, cond_dim=2):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.cond_dim = cond_dim

        # Dimensione totale dell'input (latent + condizioni)
        self.input_dim = latent_dim + cond_dim

        # Espansione iniziale
        self.fc = nn.Linear(self.input_dim, 8000)
        self.relu = nn.ReLU(inplace=True)

        # Reshape e convoluzioni
        self.conv_blocks = nn.Sequential(
            # Reshape a (batch_size, 250, 32)
            # Prima convoluzione: (batch_size, 128, 250)
            nn.Conv1d(32, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(True),

            # Espansione attraverso convoluzioni trasposte
            # (batch_size, 64, 500)
            nn.ConvTranspose1d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(True),

            # (batch_size, 32, 1000)
            nn.ConvTranspose1d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(True),

            # (batch_size, 16, 2000)
            nn.ConvTranspose1d(32, 16, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(True),

            # Output finale: (batch_size, 1, 2000)
            nn.Conv1d(16, 1, kernel_size=3, stride=1, padding=1),
            nn.Tanh()  # Normalizza l'output tra -1 e 1
        )

    def forward(self, z, cond):
        """
        Forward pass del generatore

        Args:
            z: Vettore latente (batch_size, latent_dim)
            cond: Condizionamento (batch_size, cond_dim)

        Returns:
            Accelerogramma sintetico (batch_size, 2000)
        """
        # Concatena rumore latente e condizioni
        x = torch.cat([z, cond], dim=1)

        # Espansione attraverso il layer fully connected
        x = self.fc(x)
        x = self.relu(x)

        # Reshape per convoluzione 1D: (batch_size, 32, 250)
        x = x.view(-1, 32, 250)

        # Applicazione dei blocchi convoluzionali
        x = self.conv_blocks(x)

        # Reshape finale: (batch_size, 2000)
        x = x.view(-1, 2000)

        return x

class Discriminator(nn.Module):
    """
    Discriminatore per la CGAN

    Input:
    - Segnale (originale o generato) (dimensione: 2000 campioni)
    - Informazioni condizionali (dimensione: cond_dim)

    Output:
    - Probabilità che il segnale sia reale (0-1)
    """
    def __init__(self, cond_dim=2):
        super(Discriminator, self).__init__()
        self.cond_dim = cond_dim

        # Blocchi convoluzionali per l'estrazione di feature
        self.conv_blocks = nn.Sequential(
            # Primo blocco
            nn.Conv1d(1, 32, kernel_size=7, stride=2, padding=3),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),

            # Secondo blocco
            nn.Conv1d(32, 64, kernel_size=7, stride=2, padding=3),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),

            # Terzo blocco
            nn.Conv1d(64, 128, kernel_size=7, stride=2, padding=3),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3)
        )

        # Calcolo della dimensione dopo le convoluzioni
        # Input: (1, 2000) -> Dopo 3 strati con stride=2: (128, 250)
        conv_out_size = 128 * 250

        # Fully connected finale con concatenazione delle condizioni
        self.fc1 = nn.Linear(conv_out_size + cond_dim, 64)
        self.leaky_relu = nn.LeakyReLU(0.2, inplace=True)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, cond):
        """
        Forward pass del discriminatore

        Args:
            x: Segnale (batch_size, 2000)
            cond: Condizionamento (batch_size, cond_dim)

        Returns:
            Probabilità (batch_size, 1)
        """
        # Reshape per convoluzione 1D: (batch_size, 1, 2000)
        x = x.view(-1, 1, 2000)

        # Estrazione di feature tramite convoluzioni
        x = self.conv_blocks(x)

        # Flatten
        x = x.view(x.size(0), -1)

        # Concatenazione con le condizioni
        x = torch.cat([x, cond], dim=1)

        # Fully connected finale
        x = self.fc1(x)
        x = self.leaky_relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)

        return x

# 7. Dataloader per CGAN
# @title Dataloader per CGAN

class SeismicWaveletDataset(Dataset):
    """
    Dataset che carica le componenti wavelet dei segnali sismici
    """
    def __init__(self, data_dir, transform=None, wavelet_level=6, decompose_on_fly=False):
        """
        Inizializza il dataset

        Args:
            data_dir: Directory contenente i dati normalizzati e i metadati
            transform: Trasformazioni opzionali da applicare ai dati
            wavelet_level: Livello di decomposizione wavelet
            decompose_on_fly: Se True, decompone il segnale al volo invece di caricare file precomputati
        """
        self.data_dir = data_dir
        self.transform = transform
        self.wavelet_level = wavelet_level
        self.decompose_on_fly = decompose_on_fly

        # Directory contenenti i dati
        self.normalized_dir = os.path.join(data_dir, "normalized")
        self.json_dir = os.path.join(data_dir, "json")
        self.wavelet_dir = os.path.join(data_dir, "wavelet")

        # Lista di ID disponibili
        self.aids = []
        for file in os.listdir(self.normalized_dir):
            if file.endswith('.nrm'):
                aid = os.path.splitext(file)[0]
                # Verifica che esista anche il json corrispondente
                if os.path.exists(os.path.join(self.json_dir, f"{aid}.json")):
                    self.aids.append(aid)

        print(f"Dataset wavelet inizializzato con {len(self.aids)} segnali sismici")

    def __len__(self):
        """Restituisce il numero di segnali nel dataset"""
        return len(self.aids)

    def __getitem__(self, idx):
        """
        Carica un segnale sismico, le sue componenti wavelet e i suoi metadati

        Args:
            idx: Indice del segnale

        Returns:
            Tuple (componenti_wavelet, condizioni)
            componenti_wavelet è una lista di tensori [A6, D1, D2, D3, D4, D5, D6]
        """
        aid = self.aids[idx]

        # Carica i metadati
        json_path = os.path.join(self.json_dir, f"{aid}.json")
        with open(json_path, 'r') as f:
            metadata = json.load(f)

        # Converti i metadati in vettore di condizionamento
        cond_vector = textToVect(metadata)
        cond_tensor = torch.FloatTensor(cond_vector)

        # Se decomposizione al volo
        if self.decompose_on_fly:
            # Carica il segnale normalizzato
            signal_path = os.path.join(self.normalized_dir, f"{aid}.nrm")
            signal = np.loadtxt(signal_path)

            # Decomponi il segnale
            components = wavelet_decomposition(signal, wavelet='db4', level=self.wavelet_level)
        else:
            # Carica le componenti precomputate
            components = []
            # Approssimazione
            a6_path = os.path.join(self.wavelet_dir, f"{aid}_A{self.wavelet_level}.npy")
            if os.path.exists(a6_path):
                a6 = np.load(a6_path)
                components.append(a6)
            else:
                # Se il file non esiste, usa una decomposizione al volo
                signal_path = os.path.join(self.normalized_dir, f"{aid}.nrm")
                signal = np.loadtxt(signal_path)
                components = wavelet_decomposition(signal, wavelet='db4', level=self.wavelet_level)

                # Interrompi il ciclo
                pass

            # Dettagli
            for i in range(1, self.wavelet_level + 1):
                di_path = os.path.join(self.wavelet_dir, f"{aid}_D{i}.npy")
                if os.path.exists(di_path):
                    di = np.load(di_path)
                    components.append(di)

        # Assicurati che ci siano esattamente wavelet_level + 1 componenti
        if len(components) != self.wavelet_level + 1:
            raise ValueError(f"Numero di componenti wavelet errato per AID {aid}: {len(components)}")

        # Verifica che tutte le componenti siano lunghe 2000 campioni
        for i in range(len(components)):
            if len(components[i]) != 2000:
                # Taglia o estendi
                if len(components[i]) > 2000:
                    components[i] = components[i][:2000]
                else:
                    pad_width = 2000 - len(components[i])
                    components[i] = np.pad(components[i], (0, pad_width), 'constant')

        # Applica eventuali trasformazioni
        if self.transform:
            components = [self.transform(comp) for comp in components]

        # Converte in tensori PyTorch
        component_tensors = [torch.FloatTensor(comp) for comp in components]

        return component_tensors, cond_tensor

def get_dataloader(data_dir, batch_size=16, wavelet=True, wavelet_level=6, decompose_on_fly=False, shuffle=True, num_workers=2):
    """
    Crea un DataLoader per i dati sismici

    Args:
        data_dir: Directory contenente i dati
        batch_size: Dimensione del batch
        wavelet: Se True, utilizza il dataset con decomposizione wavelet
        wavelet_level: Livello di decomposizione wavelet
        decompose_on_fly: Se True, decompone il segnale al volo invece di caricare file precomputati
        shuffle: Se True, mescola i dati
        num_workers: Numero di worker per il caricamento dati

    Returns:
        DataLoader
    """
    dataset = SeismicWaveletDataset(
        data_dir=data_dir,
        wavelet_level=wavelet_level,
        decompose_on_fly=decompose_on_fly
    )

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        drop_last=True  # Necessario per batch normalizzazione
    )

    return dataloader

# 8. Addestramento della CGAN
# @title Funzione di addestramento della CGAN

def train_cgan(data_dir, output_dir="./output", n_epochs=200, batch_size=16, latent_dim=100,
               cond_dim=2, lr=0.0002, beta1=0.5, wavelet_level=6, save_interval=10):
    """
    Addestra un modello CGAN con un singolo generatore e multi-discriminatori

    Args:
        data_dir: Directory contenente i dati
        output_dir: Directory per i risultati
        n_epochs: Numero di epoche di addestramento
        batch_size: Dimensione del batch
        latent_dim: Dimensione del vettore latente
        cond_dim: Dimensione del vettore di condizionamento
        lr: Learning rate
        beta1: Parametro beta1 per Adam
        wavelet_level: Livello di decomposizione wavelet
        save_interval: Intervallo di salvataggio del modello
    """
    # Crea directory di output
    os.makedirs(output_dir, exist_ok=True)
    models_dir = os.path.join(output_dir, "models")
    images_dir = os.path.join(output_dir, "images")
    os.makedirs(models_dir, exist_ok=True)
    os.makedirs(images_dir, exist_ok=True)

    # Imposta il device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Utilizzo device: {device}")

    # Carica i dati
    dataloader = get_dataloader(data_dir, batch_size=batch_size, wavelet=True,
                               wavelet_level=wavelet_level, decompose_on_fly=True)

    # Crea generatore e discriminatori
    generator = Generator(latent_dim, cond_dim).to(device)
    discriminators = [Discriminator(cond_dim).to(device) for _ in range(wavelet_level + 1)]

    # Inizializza i pesi
    generator.apply(weights_init)
    for disc in discriminators:
        disc.apply(weights_init)

    # Setup ottimizzatori
    optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))
    optimizer_Ds = [
        optim.Adam(disc.parameters(), lr=lr, betas=(beta1, 0.999))
        for disc in discriminators
    ]

    # Criterio di perdita
    criterion = nn.BCELoss()

    # Pesi uniformi iniziali per i discriminatori
    alphas = np.ones(wavelet_level + 1) / (wavelet_level + 1)

    # Log per tenere traccia dei progressi
    losses_G = []
    losses_D = []

    print("Inizio addestramento...")
    start_time = time.time()

    for epoch in range(n_epochs):
        for i, (components, conditions) in enumerate(dataloader):
            batch_size_actual = conditions.size(0)

            # Verifica che ci siano dati sufficienti per procedere
            if batch_size_actual == 0:
                print("Batch vuoto, skippo")
                continue

            # Preparazione delle etichette
            real_label = torch.ones(batch_size_actual, 1, device=device)
            fake_label = torch.zeros(batch_size_actual, 1, device=device)

            # Passa le condizioni al device corretto
            conditions = conditions.to(device)

            # =============================================================
            # (1) Aggiorna i discriminatori: max log(D(x)) + log(1 - D(G(z)))
            # =============================================================
            # Addestra ogni discriminatore separatamente
            d_losses = []
            for j, discriminator in enumerate(discriminators):
                optimizer_Ds[j].zero_grad()

                # Verifica che ci siano abbastanza componenti
                if j >= len(components):
                    print(f"Avviso: Mancano componenti per il discriminatore {j}")
                    d_losses.append(0.0)  # Aggiungi un valore fittizio alla perdita
                    continue

                # Carica la componente wavelet j-esima
                real_comp = components[j].to(device)

                # Verifica che le dimensioni siano corrette
                if real_comp.dim() == 1:
                    real_comp = real_comp.unsqueeze(0)  # Aggiungi dimensione batch se mancante

                # Calcola output del discriminatore con dati reali
                output_real = discriminator(real_comp, conditions)
                d_loss_real = criterion(output_real, real_label)

                # Genera dati falsi
                z = torch.randn(batch_size_actual, latent_dim, device=device)
                fake_signal = generator(z, conditions)

                # Decomponi il segnale falso in componenti wavelet
                # Usa il CPU per la decomposizione wavelet (più compatibile)
                # Importante: usa detach() per staccare dal grafo computazionale
                fake_signal_np = fake_signal.detach().cpu().numpy()  # Aggiunto detach()

                # Decomponi ogni segnale individualmente nel batch
                fake_comps_list = []
                for k in range(fake_signal_np.shape[0]):
                    comps = wavelet_decomposition(fake_signal_np[k], level=wavelet_level)
                    fake_comps_list.append(comps[j])  # Prendi solo la componente j-esima

                # Converti la lista in un tensore batch
                fake_comp_np = np.stack(fake_comps_list)
                fake_comp = torch.tensor(fake_comp_np, device=device, dtype=torch.float32)

                # Calcola output del discriminatore con dati falsi
                output_fake = discriminator(fake_comp, conditions)
                d_loss_fake = criterion(output_fake, fake_label)

                # Perdita totale per il discriminatore j-esimo
                d_loss = d_loss_real + d_loss_fake
                d_loss.backward()
                optimizer_Ds[j].step()

                d_losses.append(d_loss.item())

            # =============================================================
            # (2) Aggiorna il generatore: max log(D(G(z)))
            # =============================================================
            optimizer_G.zero_grad()

            # Genera nuovi dati falsi
            z = torch.randn(batch_size_actual, latent_dim, device=device)
            fake_signal = generator(z, conditions)

            # Decomponi nuovamente il segnale falso
            # Usa il CPU per la decomposizione wavelet
            # Qui NON usare detach() perché abbiamo bisogno dei gradienti
            fake_signal_np = fake_signal.cpu().detach().numpy()  # Aggiunto detach()

            # Perdita del generatore complessiva
            g_loss = 0

            # Calcola perdita per ogni discriminatore
            for j, discriminator in enumerate(discriminators):
                if j >= wavelet_level + 1:
                    continue  # Skip se indice fuori range

                # Decomponi ogni segnale nel batch
                fake_comps_list = []
                for k in range(fake_signal_np.shape[0]):
                    comps = wavelet_decomposition(fake_signal_np[k], level=wavelet_level)
                    if j < len(comps):
                        fake_comps_list.append(comps[j])
                    else:
                        # Se per qualche motivo la decomposizione non ha abbastanza componenti
                        print(f"Avviso: Decomposizione incompleta per indice {j}")
                        # Crea una componente vuota
                        fake_comps_list.append(np.zeros(2000))

                # Converti la lista in un tensore batch
                fake_comp_np = np.stack(fake_comps_list)
                fake_comp = torch.tensor(fake_comp_np, device=device, dtype=torch.float32)

                # Il generatore vuole che il discriminatore classifichi i segnali come reali
                output = discriminator(fake_comp, conditions)
                component_loss = criterion(output, real_label)
                g_loss += alphas[j] * component_loss

            # Aggiungi anche il loss diretto (senza decomposizione)
            direct_output = discriminators[0](fake_signal, conditions)
            direct_loss = criterion(direct_output, real_label)
            g_loss += direct_loss

            # Backpropagation
            g_loss.backward()
            optimizer_G.step()

            # =============================================================
            # Calcola nuovi pesi alpha basati sull'energia delle componenti
            # =============================================================
            if i % 10 == 0:  # Aggiorna i pesi ogni 10 batch
                # Prendi un batch di segnali reali e calcola le energie
                try:
                    # Seleziona un campione casuale di componenti
                    sample_idx = np.random.randint(0, batch_size_actual, min(4, batch_size_actual))
                    energies = []

                    for idx in sample_idx:
                        # Prendi tutte le componenti per questo campione
                        real_comps = [comp[idx].cpu().numpy() for comp in components]
                        batch_energies = get_component_energy(real_comps)
                        energies.append(batch_energies)

                    # Media le energie su tutti i batch campionati
                    if energies:
                        mean_energies = np.mean(energies, axis=0)
                        # Assicurati che ci siano esattamente wavelet_level + 1 pesi
                        if len(mean_energies) == wavelet_level + 1:
                            alphas = mean_energies
                except Exception as e:
                    print(f"Errore nel calcolo delle energie: {str(e)}")

            # =============================================================
            # Log
            # =============================================================
            if i % 10 == 0:
                elapsed = time.time() - start_time
                d_loss_mean = np.mean([l for l in d_losses if l > 0])  # Media solo delle perdite valide
                print(f"[{epoch}/{n_epochs}][{i}/{len(dataloader)}] "
                      f"Loss_D: {d_loss_mean:.4f} Loss_G: {g_loss.item():.4f} "
                      f"Time: {elapsed:.2f}s")

                # Salva le perdite per il plot
                losses_G.append(g_loss.item())
                losses_D.append(d_loss_mean)

                # Genera e salva un esempio
                with torch.no_grad():
                    # Genera un singolo esempio
                    z = torch.randn(1, latent_dim, device=device)
                    # Usa la prima condizione del batch come esempio
                    fixed_cond = conditions[0].unsqueeze(0) if batch_size_actual > 0 else torch.tensor([[0.33, 0.0]], device=device)

                    fake = generator(z, fixed_cond).cpu().numpy()[0]

                    # Plot
                    plt.figure(figsize=(10, 4))
                    plt.plot(fake)
                    plt.title(f"Accelerogramma sintetico - Epoca {epoch}")
                    plt.xlabel("Campioni")
                    plt.ylabel("Accelerazione (normalizzata)")
                    plt.savefig(os.path.join(images_dir, f"epoch_{epoch:03d}_batch_{i:04d}.png"))
                    plt.close()

        # Salva il modello periodicamente
        if (epoch + 1) % save_interval == 0 or epoch == n_epochs - 1:
            try:
                torch.save({
                    'epoch': epoch,
                    'generator_state_dict': generator.state_dict(),
                    'discriminator_state_dicts': [disc.state_dict() for disc in discriminators],
                    'optimizer_G_state_dict': optimizer_G.state_dict(),
                    'optimizer_Ds_state_dict': [opt.state_dict() for opt in optimizer_Ds],
                    'losses_G': losses_G,
                    'losses_D': losses_D,
                    'alphas': alphas.tolist() if isinstance(alphas, np.ndarray) else alphas
                }, os.path.join(models_dir, f"model_epoch_{epoch:03d}.pt"))

                # Salva anche l'ultimo modello
                torch.save({
                    'epoch': epoch,
                    'generator_state_dict': generator.state_dict(),
                    'discriminator_state_dicts': [disc.state_dict() for disc in discriminators],
                    'optimizer_G_state_dict': optimizer_G.state_dict(),
                    'optimizer_Ds_state_dict': [opt.state_dict() for opt in optimizer_Ds],
                    'losses_G': losses_G,
                    'losses_D': losses_D,
                    'alphas': alphas.tolist() if isinstance(alphas, np.ndarray) else alphas
                }, os.path.join(models_dir, "latest_model.pt"))
            except Exception as e:
                print(f"Errore nel salvataggio del modello: {str(e)}")

    # Plot finale delle perdite
    try:
        plt.figure(figsize=(10, 5))
        plt.plot(losses_G, label='Generator')
        plt.plot(losses_D, label='Discriminator')
        plt.xlabel('Iterations (x50)')
        plt.ylabel('Loss')
        plt.legend()
        plt.title('Training Losses')
        plt.savefig(os.path.join(output_dir, "training_losses.png"))
        plt.close()
    except Exception as e:
        print(f"Errore nella creazione del plot delle perdite: {str(e)}")

    print("Addestramento completato!")
    return generator, discriminators

# 9. Generazione e valutazione di accelerogrammi sintetici
# @title Funzioni per generazione e valutazione

def load_model(model_path, latent_dim=100, cond_dim=2):
    """
    Carica un modello pre-addestrato

    Args:
        model_path: Percorso del file del modello
        latent_dim: Dimensione del vettore latente
        cond_dim: Dimensione del vettore di condizionamento

    Returns:
        Generatore caricato
    """
    # Crea il generatore
    generator = Generator(latent_dim, cond_dim)

    # Carica i pesi del modello con weights_only=False per compatibilità
    checkpoint = torch.load(model_path, map_location=torch.device('cpu'), weights_only=False)
    generator.load_state_dict(checkpoint['generator_state_dict'])

    # Imposta il generatore in modalità di valutazione (non addestramento)
    generator.eval()

    return generator

def generate_accelerograms(generator, n_samples=10, conditions=None, output_dir=None, latent_dim=100):
    """
    Genera accelerogrammi sintetici utilizzando un generatore addestrato

    Args:
        generator: Generatore addestrato
        n_samples: Numero di campioni da generare
        conditions: Lista di coppie (mw, pga) da utilizzare per il condizionamento
                   Se None, utilizza valori casuali
        output_dir: Directory dove salvare i risultati
        latent_dim: Dimensione del vettore latente

    Returns:
        Lista di accelerogrammi sintetici
    """
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    device = next(generator.parameters()).device

    # Se non vengono fornite condizioni, genera valori casuali
    if conditions is None:
        # Genera magnitudo tra 4.0 e 7.0
        mw_values = np.random.uniform(4.0, 7.0, n_samples)

        # Genera PGA tra 0.05 e 0.5 g
        pga_values = np.random.uniform(0.05, 0.5, n_samples)

        conditions = list(zip(mw_values, pga_values))

    # Genera gli accelerogrammi
    synthetic_signals = []

    with torch.no_grad():
        for i, (mw, pga) in enumerate(conditions):
            print(f"Generazione accelerogramma {i+1}/{len(conditions)} con Mw={mw:.1f}, PGA={pga:.2f}g")

            # Crea dizionario per textToVect
            metadata = {'MW': mw, 'PGA': pga}

            # Converti in vettore di condizionamento
            cond_vect = textToVect(metadata)
            cond_tensor = torch.FloatTensor(cond_vect).unsqueeze(0).to(device)

            # Genera rumore casuale
            z = torch.randn(1, latent_dim, device=device)

            # Genera l'accelerogramma
            fake_signal = generator(z, cond_tensor).cpu().numpy()[0]
            synthetic_signals.append(fake_signal)

            # Salva l'accelerogramma
            if output_dir:
                # Salva i dati del segnale
                signal_path = os.path.join(output_dir, f"synthetic_{i+1:03d}.npy")
                np.save(signal_path, fake_signal)

                # Salva i metadati
                metadata_path = os.path.join(output_dir, f"synthetic_{i+1:03d}.json")
                metadata_obj = {
                    'id': f"synthetic_{i+1:03d}",
                    'mw': float(mw),
                    'pga': float(pga)
                }
                with open(metadata_path, 'w') as f:
                    json.dump(metadata_obj, f, indent=4)

                # Crea e salva il plot
                plt.figure(figsize=(10, 4))
                t = np.arange(len(fake_signal)) / 200  # Assume 200 Hz sampling rate
                plt.plot(t, fake_signal)
                plt.title(f"Accelerogramma sintetico (Mw={mw:.1f}, PGA={pga:.2f}g)")
                plt.xlabel("Tempo (s)")
                plt.ylabel("Accelerazione (normalizzata)")
                plt.grid(True)
                plt.savefig(os.path.join(output_dir, f"synthetic_{i+1:03d}.png"))
                plt.close()

                # Opzionalmente, genera anche la decomposizione wavelet
                components = wavelet_decomposition(fake_signal, level=6)

                # Crea plot delle componenti
                plt.figure(figsize=(12, 15))

                # Plot del segnale originale
                plt.subplot(8, 1, 1)
                plt.plot(t, fake_signal)
                plt.title(f"Accelerogramma sintetico (Mw={mw:.1f}, PGA={pga:.2f}g)")
                plt.ylabel("Ampiezza")
                plt.grid(True)

                # Plot delle componenti
                labels = ['A6', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6']
                for j, comp in enumerate(components):
                    if j < len(labels):
                        plt.subplot(8, 1, j+2)
                        plt.plot(t, comp)
                        plt.title(f"Componente {labels[j]}")
                        plt.ylabel("Ampiezza")
                        plt.grid(True)

                plt.tight_layout()
                plt.savefig(os.path.join(output_dir, f"synthetic_{i+1:03d}_wavelet.png"))
                plt.close()

    return synthetic_signals

def evaluate_accelerograms(synthetic_signals, output_dir=None):
    """
    Valuta gli accelerogrammi sintetici

    Args:
        synthetic_signals: Lista di accelerogrammi sintetici
        output_dir: Directory dove salvare i risultati

    Returns:
        Dizionario con le metriche di valutazione
    """
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    metrics = {}

    # Analisi statistica dei segnali sintetici
    amplitudes = np.concatenate([signal for signal in synthetic_signals])

    metrics['mean'] = float(np.mean(amplitudes))
    metrics['std'] = float(np.std(amplitudes))
    metrics['min'] = float(np.min(amplitudes))
    metrics['max'] = float(np.max(amplitudes))
    metrics['median'] = float(np.median(amplitudes))

    # Crea un istogramma delle ampiezze
    plt.figure(figsize=(10, 6))
    plt.hist(amplitudes, bins=50, alpha=0.7, color='blue')
    plt.title('Distribuzione delle ampiezze degli accelerogrammi sintetici')
    plt.xlabel('Ampiezza')
    plt.ylabel('Frequenza')
    plt.grid(True)

    if output_dir:
        plt.savefig(os.path.join(output_dir, 'amplitude_distribution.png'))
    plt.close()

    # Analisi spettrale
    plt.figure(figsize=(12, 8))

    # Calcolo dello spettro di potenza medio
    psd_all = []

    for i, signal in enumerate(synthetic_signals):
        # Calcola la trasformata di Fourier
        fft_result = np.fft.fft(signal)

        # Calcola le frequenze corrispondenti (assumendo 200 Hz di campionamento)
        n = len(signal)
        fs = 200  # Hz
        freqs = np.fft.fftfreq(n, 1/fs)

        # Calcola la densità spettrale di potenza (PSD)
        psd = np.abs(fft_result)**2 / n

        # Considera solo le frequenze positive
        positive_freqs = freqs[1:n//2]
        positive_psd = psd[1:n//2]

        # Aggiungi alla lista
        psd_all.append(positive_psd)

        # Plot individuale (solo alcuni per leggibilità)
        if i < 5:  # Mostra solo i primi 5 spettri
            plt.semilogy(positive_freqs, positive_psd, alpha=0.3,
                      label=f'Segnale sintetico {i+1}')

    # Calcola e plotta lo spettro medio
    if psd_all:
        mean_psd = np.mean(psd_all, axis=0)
        plt.semilogy(positive_freqs, mean_psd, 'k-', linewidth=2,
                  label='Media dei segnali sintetici')

    plt.title('Analisi spettrale degli accelerogrammi sintetici')
    plt.xlabel('Frequenza (Hz)')
    plt.ylabel('Densità spettrale di potenza')
    plt.legend()
    plt.grid(True)

    if output_dir:
        plt.savefig(os.path.join(output_dir, 'spectral_analysis.png'))
    plt.close()

    # Salva le metriche
    if output_dir:
        with open(os.path.join(output_dir, 'evaluation_metrics.json'), 'w') as f:
            json.dump(metrics, f, indent=4)

    return metrics

# 10. Interfaccia principale
# @title Caricamento del file CSV con dati reali
print("Per favore, carica il file CSV contenente i dati degli eventi sismici reali.")
print("Il file deve contenere le colonne: esm_event_id, station_code, mw, epi_dist")
print("Formato CSV con separatore punto e virgola (;)")

# Carica il file CSV con i dati reali
csv_upload = files.upload()

if len(csv_upload) == 0:
    raise ValueError("Nessun file caricato. È necessario caricare un file CSV per procedere.")

# Ottieni il nome del primo file caricato
csv_filename = list(csv_upload.keys())[0]

# Verifica che sia un file CSV
if not csv_filename.endswith('.csv'):
    raise ValueError(f"Il file caricato ({csv_filename}) non è un file CSV.")

# Copia il file nella directory corrente
with open(csv_filename, 'wb') as f:
    f.write(csv_upload[csv_filename])

print(f"File CSV caricato con successo: {csv_filename}")

# Verifica il contenuto del file
try:
    df = pd.read_csv(csv_filename, sep=';')
    required_columns = ['esm_event_id', 'station_code', 'mw', 'epi_dist']
    missing_columns = [col for col in required_columns if col not in df.columns]

    if missing_columns:
        print(f"ATTENZIONE: Il file manca delle seguenti colonne: {', '.join(missing_columns)}")
    else:
        print(f"Il file contiene {len(df)} eventi sismici.")
        print("Prime righe del file:")
        print(df.head())
except Exception as e:
    print(f"Errore nella lettura del file CSV: {str(e)}")

# 11. Flusso di lavoro principale
# @title Esecuzione del flusso di lavoro
execute_download = True  # @param {type:"boolean"}
execute_wavelet = True  # @param {type:"boolean"}
execute_training = True  # @param {type:"boolean"}
execute_generation = True  # @param {type:"boolean"}
n_epochs = 50  # @param {type:"slider", min:10, max:500, step:10}
batch_size = 8  # @param {type:"slider", min:1, max:32, step:1}

# Step 1: Download e preprocessamento dei dati
if execute_download:
    print("\n====== FASE 1: Download e preprocessamento dei dati ======")
    # Usa il file CSV caricato dall'utente
    csv_path = os.path.join(os.getcwd(), csv_filename)
    download_and_process_data(csv_path, DATA_DIR)

# Step 2: Decomposizione wavelet
if execute_wavelet:
    print("\n====== FASE 2: Decomposizione wavelet ======")
    process_all_signals(DATA_DIR, wavelet='db4', level=6, plot=True)

# Step 3: Addestramento del modello CGAN
if execute_training:
    print("\n====== FASE 3: Addestramento del modello CGAN ======")
    generator, discriminators = train_cgan(
        data_dir=DATA_DIR,
        output_dir=OUTPUT_DIR,
        n_epochs=n_epochs,
        batch_size=batch_size,
        latent_dim=100,
        lr=0.0002,
        wavelet_level=6,
        save_interval=5
    )

# Step 4: Generazione e valutazione di accelerogrammi sintetici
if execute_generation:
    print("\n====== FASE 4: Generazione e valutazione di accelerogrammi sintetici ======")
    model_path = os.path.join(OUTPUT_DIR, "models", "latest_model.pt")

    if os.path.exists(model_path):
        # Carica il modello addestrato
        generator = load_model(model_path)

        # Estrai i valori di magnitudo dai dati reali per usarli come condizioni
        try:
            df = pd.read_csv(csv_path, sep=';')
            magnitudes = df['mw'].unique()

            # Crea condizioni basate sui dati reali
            conditions = []
            for mw in magnitudes:
                # Aggiungi diverse combinazioni di PGA per ogni magnitudo
                pga_values = [0.1, 0.2, 0.3]  # Valori PGA di esempio
                for pga in pga_values:
                    conditions.append((float(mw), pga))

            # Limita a 10 condizioni per non generare troppi segnali
            conditions = conditions[:10]

            print(f"Generazione di accelerogrammi con le seguenti condizioni:")
            for mw, pga in conditions:
                print(f"Mw: {mw}, PGA: {pga}g")
        except Exception as e:
            print(f"Errore nell'estrazione delle magnitudo dai dati reali: {str(e)}")
            print("Utilizzo di condizioni predefinite...")

            # Condizioni predefinite in caso di errore
            conditions = [
                (4.5, 0.10),
                (5.0, 0.15),
                (5.5, 0.20),
                (6.0, 0.25),
                (6.5, 0.30),
                (7.0, 0.40),
            ]

        # Directory di output per gli accelerogrammi generati
        generated_dir = os.path.join(OUTPUT_DIR, "generated")
        evaluation_dir = os.path.join(OUTPUT_DIR, "evaluation")

        # Genera gli accelerogrammi
        print("Generazione accelerogrammi sintetici...")
        synthetic_signals = generate_accelerograms(
            generator,
            n_samples=len(conditions),
            conditions=conditions,
            output_dir=generated_dir
        )

        # Valuta gli accelerogrammi
        print("Valutazione accelerogrammi sintetici...")
        metrics = evaluate_accelerograms(
            synthetic_signals,
            output_dir=evaluation_dir
        )

        print("Metriche di valutazione:")
        for key, value in metrics.items():
            print(f"  {key}: {value}")
    else:
        print(f"ERRORE: Modello non trovato in {model_path}. Esegui prima l'addestramento.")

print("\nProcesso completato!")

Dispositivo utilizzato: cpu
Dispositivo utilizzato: cpu
Per favore, carica il file CSV contenente i dati degli eventi sismici reali.
Il file deve contenere le colonne: esm_event_id, station_code, mw, epi_dist
Formato CSV con separatore punto e virgola (;)


Saving dati.csv to dati.csv
File CSV caricato con successo: dati.csv
Il file contiene 34 eventi sismici.
Prime righe del file:
            esm_event_id           event_time  ingv_event_id  ev_latitude  \
0  EMSC-20161030_0000029  2016-10-30T06:40:18      8863681.0     42.83794   
1  EMSC-20161030_0000029  2016-10-30T06:40:18      8863681.0     42.83794   
2  EMSC-20161030_0000029  2016-10-30T06:40:18      8863681.0     42.83794   
3  EMSC-20161030_0000029  2016-10-30T06:40:18      8863681.0     42.83794   
4  EMSC-20161030_0000029  2016-10-30T06:40:18      8863681.0     42.83794   

   ev_longitude  ev_depth_km              ev_hyp_ref fm_type_code  \
0      13.12324        6.169  Spallarossa_et_al_2021           NF   
1      13.12324        6.169  Spallarossa_et_al_2021           NF   
2      13.12324        6.169  Spallarossa_et_al_2021           NF   
3      13.12324        6.169  Spallarossa_et_al_2021           NF   
4      13.12324        6.169  Spallarossa_et_al_2021           NF